# 导入模型

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_ollama import ChatOllama
from langchain_huggingface import HuggingFaceEmbeddings  # HuggingFace嵌入

load_dotenv(dotenv_path=r"E:\Workplace\GitHub\Learning\LangChain\.env", override=True)

model = ChatOpenAI(
                    temperature=0.1,
                    max_tokens=2048,
                    presence_penalty=1.2,
                    # model='deepseek-chat',
                    # openai_api_key=os.getenv('DEEPSEEK_API_KEY'),
                    # openai_api_base="https://api.deepseek.com/v1",
                    # model='gpt-4o-mini',
                    # openai_api_key=os.getenv('CLOSEAI_API_KEY'),
                    # openai_api_base="https://api.openai-proxy.org/v1",
                    # model='yi-lightning',
                    # openai_api_key=os.getenv('YI_API_KEY'),
                    # openai_api_base="https://api.lingyiwanwu.com/v1",
                    model='ernie-4.0-turbo-8k-latest',
                    openai_api_key=os.getenv('BD_API_KEY'),
                    openai_api_base="https://qianfan.baidubce.com/v2",
                    default_headers={"appid": "app-dz4Hp7OK"}
                )

# model = ChatOllama(
#                     temperature=0,
#                     num_predict=4096,
#                     repeat_penalty=1.2,
#                     model="llama3.2:3b"
#                 )

embeddings = HuggingFaceEmbeddings(
            model_name=r"E:/Workplace/GitHub/Embedding/maidalun/bce-embedding-base_v1",
            model_kwargs={"device": "cuda",
                          "trust_remote_code": True},
            encode_kwargs={"normalize_embeddings": True},
        )

c:\Users\farben\anaconda3\envs\lclg\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 文档加载器

In [2]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader(
    file_path="三国演义.txt",
    autodetect_encoding=True,
)

docs = loader.load()

docs

[Document(metadata={'source': '三国演义.txt'}, page_content='\n本书由爱上阅读的网友搜集整理，更多精彩书籍请访问https://www.isyd.net\n声明：本书来自互联网，仅供参考、查阅资料，版权归作者所有！！\n敬告：请在下载后的24小时内删除，同时不得用于商业用途。\n书名：三国演义\n作者：罗贯中\n\n章节数：162\n字数：647773\n========简介========\n《三国演义》反映了丰富的历史内容，人物名称、地理名称、主要事件与《三国志》基本相同。人物性格也是在《三国志》留下的固定形象基础上，才进行再发挥，进行夸张、美化、丑化等等，这也是历史演义小说的套路。《三国演义》一方面反映了较为真实的三国历史，照顾到读者希望了解真实历史的需要；另一方面，根据明朝社会的实际情况对三国人物进行了夸张、美化、丑化等等。\n\n========正文========\n第一回 桃园结义\n东汉末年，朝政腐败，再加上连年灾荒，老百姓的日子非常困苦。巨鹿人张角见人民怨恨官府，便与他的张梁、张宝在河北、河南、山东、湖北、江苏等地，招收了五十万人，举行起义，一起向官兵进攻。\n没有几天，四方百姓，头裹黄巾，跟随张角三兄弟杀向官府，声势非常浩大。汉灵帝得到各地报告，连忙下令各地官军防备。又派中郎将卢植、皇甫嵩、朱隽率领精兵，分路攻打张角兄弟的黄巾军。\n张角领军攻打幽州地界，幽州太守连忙召校尉邹靖商议，邹靖说幽州兵少不能抵挡。建议写榜文到各县招募兵马。\n榜文行到涿县，引出一名英雄，这人姓刘名备，字玄德。因家里贫寒，靠贩麻鞋、织草席为生。这天他进城来看榜文。\n刘备看完了榜文，不觉感慨地长叹了一声。忽听身后有个人大声喝道∶“大丈夫不给国家出力，叹什么气？”\n刘备回头一看，这人身高八尺，豹子头，圆眼睛，满腮的胡须像钢丝一样竖着，声音像洪钟，样子十分威武。那人对刘备说他姓张名飞，字翼德，做着卖酒、屠宰猪羊的生意。他愿意拿出家产作本钱，与刘备共同干一番大事业。\n刘备、张飞两人谈得投机，便一起到村口的一家酒店饮酒叙话。\n这时，一推车大汉进店饮酒。刘备留神一看，这人有九尺高，胸前长须飘飘，脸色好像红枣一样，长一双丹凤眼，两条卧蚕眉，相貌非常威武雄伟。刘备连忙起身，邀他过来同坐，并请问姓名。\n那人说∶“我姓关名羽，字

# 文档分割

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    separators=["。|！|？", "\.\s|\!\s|\?\s", "；|;\s", "，|,\s"],  # 定义分割符
    keep_separator="end",
    is_separator_regex=True,
    chunk_size=500,  # 每块大小
    chunk_overlap=0,  # 重叠大小
)

In [4]:
chunks = splitter.split_documents(docs)

print(len(chunks))
# chunks

106


# 向量化并存入向量库

In [5]:
from langchain_milvus.vectorstores.milvus import Milvus

db = Milvus(
        embedding_function=embeddings,
        collection_name="sg",
        connection_args={"uri": "https://in05-e9b7822ca8ff8bb.serverless.ali-cn-hangzhou.cloud.zilliz.com.cn",
                         "token": "7dc031089e6d2095e90ddb01b22837bb68f444658e6efdb726899623178d777b307be7a264fb8014b445786993ce898d7e93e486"},
        drop_old=True,
        auto_id=True,
        enable_dynamic_field=True
    )

In [6]:
db.add_documents(chunks)

c:\Users\farben\anaconda3\envs\lclg\Lib\site-packages\transformers\models\xlm_roberta\modeling_xlm_roberta.py:371: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


[454854030245099843,
 454854030245099844,
 454854030245099845,
 454854030245099846,
 454854030245099847,
 454854030245099848,
 454854030245099849,
 454854030245099850,
 454854030245099851,
 454854030245099852,
 454854030245099853,
 454854030245099854,
 454854030245099855,
 454854030245099856,
 454854030245099857,
 454854030245099858,
 454854030245099859,
 454854030245099860,
 454854030245099861,
 454854030245099862,
 454854030245099863,
 454854030245099864,
 454854030245099865,
 454854030245099866,
 454854030245099867,
 454854030245099868,
 454854030245099869,
 454854030245099870,
 454854030245099871,
 454854030245099872,
 454854030245099873,
 454854030245099874,
 454854030245099875,
 454854030245099876,
 454854030245099877,
 454854030245099878,
 454854030245099879,
 454854030245099880,
 454854030245099881,
 454854030245099882,
 454854030245099883,
 454854030245099884,
 454854030245099885,
 454854030245099886,
 454854030245099887,
 454854030245099888,
 454854030245099889,
 454854030245

# 导入提示词

In [7]:
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")

In [8]:
prompt.invoke({"context": "三国演义", 
               "question": "刘备是谁？"})

ChatPromptValue(messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: 刘备是谁？ \nContext: 三国演义 \nAnswer:", additional_kwargs={}, response_metadata={})])

# LangGraph Agent来实现

In [21]:
from langgraph.graph import MessagesState, StateGraph
from langgraph.prebuilt import ToolNode
from langchain_core.messages import SystemMessage
from langchain_core.tools import tool

@tool(response_format="content_and_artifact")
def retrieve(query: str):
    """Retrieve information related to a query."""
    retrieved_docs = db.similarity_search_with_relevance_scores(query, k=4)
    serialized = "\n\n".join(
        (f"Source: {doc[0].metadata}\n" f"Content: {doc[0].page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

def query_or_respond(state: MessagesState):
    """Generate tool call for retrieval or respond."""
    llm_with_tools = model.bind_tools([retrieve])
    response = llm_with_tools.invoke(state["messages"])
    # MessagesState appends messages to state instead of overwriting
    return {"messages": [response]}

In [23]:
res = query_or_respond({"messages": [("user", "刘备是谁？")]})
res

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '1a0eb5f67828c000', 'function': {'arguments': '{"query": "刘备生平"}', 'name': 'retrieve'}, 'type': 'function'}, {'id': '1a0eb5f67828c001', 'function': {'arguments': '{"query": "刘备主要成就"}', 'name': 'retrieve'}, 'type': 'function'}, {'id': '1a0eb5f67828c002', 'function': {'arguments': '{"query": "刘备时代背景"}', 'name': 'retrieve'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 3, 'total_tokens': 28, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'ernie-4.0-turbo-8k-latest', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-0c666745-f8ec-4e29-a1e3-43fc279422f4-0', tool_calls=[{'name': 'retrieve', 'args': {'query': '刘备生平'}, 'id': '1a0eb5f67828c000', 'type': 'tool_call'}, {'name': 'retrieve', 'args': {'query': '刘备主要成就'}, 'id': '1a0eb5f67828c001', 'type': 'tool_call'}, {'name': 'retrie